> Tah Plen

for each feature and each label...
- model (simplify) the data to known distributions (gaussian, etc)
- calculate probability that input data (single feature) fits to given distribution

analyzing probability through multiple feature and label are expected to give prediction in method such as... (what should be done after this)
- for each feature, calculate each probability where it belongs to 0 or 1 and multiply them for each class
- assume the class only based on if probability exceed certain threshold for certain class
- and more

so for this code,

when making a model...

input requires two lists as below

    [list of values in single feature with class 0], [" with class 1] (, might have more inputs to define model like defining (assuming) distribution)

and output will be a 
    
    list/dictionary that summerizes the distribution

(it will feed directly to below without going anywhere, so i'll take this on my own)

when making a prediction...

input includes
    
    data to predict, a summarized model

output will be a list with two elements

    [probability of data included in class 0, " class 1]

note about bayes

P( has specific lable | has specific value ) = P( has specific value | has specific label ) * P( has specific label ) / P( has specific value )

- interest: P( label | value )


- P( value | label ): from modeling the data distribution with specific label
- P( label ): from ratio
- P( value ): from modeling the whole data distribution

In [3]:
import math

def mean(numbers: list) -> float:
    """Bayes: Calculate the mean of a list of numbers
    
    Requires math library
    
    from machinelearningmastery"""
    
    return sum(numbers)/float(len(numbers))

def stdev(numbers: list) -> float:
    """Bayes: Calculate the standard deviation of a list of numbers
    
    Requires math library
    
    from machinelearningmastery"""
    avg = mean(numbers)
    variance = sum([(x-avg)**2 for x in numbers]) / float(len(numbers)-1)
    return math.sqrt(variance)

def summarize_gaussian(numbers: list) -> dict:
    """Bayes: Calculate the mean, stdev, and len for numbers which is a summerized gaussian model
    
    Requires math library and means, stdev function
    
    modified from machinelearningmastery"""
    summaries = {'model': 'gaussian', 'mean': mean(numbers), 'stdev': stdev(numbers), 'len': len(numbers)}
    return summaries

In [2]:
def calculate_probability_gaussian(x: float, mean: float, stdev: float) -> float:
    """Bayes: Calculate the Gaussian probability distribution function for x
    
    Requires math library
    
    from machinelearningmastery"""
    exponent = math.exp(-((x-mean)**2 / (2 * stdev**2 )))
    return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent
 
def calculate_class_probabilities(target: float, total_summary: dict, label0_summary: dict, label1_summary: dict) -> tuple:
    """Bayes: Calculate the probabilities of target belong in label0 and label1
    
    Requires calculate_probability_gaussian function"""
    # if model != gaussian -> return -1
    px0 = calculate_probability_gaussian(target, label0_summary['mean'], label0_summary['stdev'])
    px1 = calculate_probability_gaussian(target, label1_summary['mean'], label1_summary['stdev'])
    p0 = label0_summary['len'] / total_summary['len']
    p1 = label1_summary['len'] / total_summary['len']
    px = calculate_probability_gaussian(target, total_summary['mean'], total_summary['stdev'])
    p0x = px0 * p0 / px
    p1x = px1 * p1 / px
    return p0x, p1x

In [1]:
def thatfunctionthatconsiderspossibilityofmultiplefeatureandchoosetheclasswhichgivendatabelongstoandalsohascomicallylongname(probabilities: list, debug: int = 0) -> int: # bias: list = [],
    """Bayes: Calculates combinations of possibilities that certain value belongs to each features while not others
    
    Add each probability for when each class is dominant and compare the value in order to predict the label
    
    probabilities
        list of tuples from calculate_class_probabilities
    
    debug
        when value is 1, print the values of each possibilities and other detailed statistics
    """
    
    # 예시: 5개의 feature 중 예측된 레이블이 각 feature 순서대로 첫 2개에서 0, 뒷 3개에서 1이라고 예측했을 때 실제 값이 0일 확률은
    # (1번 레이블에서 0이라고 예측한 게 맞을 확률) * (2번 ") * (3번 레이블에서 1이라고 예측한 게 틀릴 확률) * ...
    # 틀릴 확률 = 1 - 맞을 확률
    
    # bias (X)
    #     Use to multiply the consideration to certain feature
        
    #     when [1, 1, 2, 1, 1], third feature will be considered twice than other features
    
    iszero = 0
    isone = 0
    featurecount = len(probabilities)
    
    # if len(bias) != featurecount:
    #     bias = [1] * featurecount
    
    # if debug == 1:
    #     for b in range(len(bias)):
    #         if b != 1:
    #             print(f"bias in feature #{b}: x{bias[b]}")
    
    for i in range(0, 2**featurecount):
        currentcombination = eval("'{:0>" + str(featurecount) + "}'.format(bin(" + str(i) + ")[2:])")
        iscurrentcombinationofzero = 1
        iscurrentcombinationofone = 1
        for j in range(featurecount):
            if currentcombination[j] == '0':
                iscurrentcombinationofzero *= probabilities[j][0] # * bias[j]
                iscurrentcombinationofone *= (1 - probabilities[j][0])
            else:
                iscurrentcombinationofzero *= (1 - probabilities[j][1]) # * bias[j]
                iscurrentcombinationofone *= probabilities[j][1]
        iszero += iscurrentcombinationofzero
        isone += iscurrentcombinationofone
        if debug:
            print(f"combination: {currentcombination} | probability of label being 0 {iscurrentcombinationofzero}")
            print(" " * len(f"combination: {currentcombination} ") + f"| probability of label being 0 {iscurrentcombinationofone}")
    if debug:
        print(f"total probability of being 0: {iszero}")
        print(f"total probability of being 1: {isone}")
    if iszero >= isone:
        return 0
    else:
        return 1

In [ ]:
def summerize_multiple_features(featured: list) -> list:
    """Bayes: summerize each feature in featured with gaussian and return list of summerized models
    
    Requires summerize gaussian function
    
    elements of returned list is a list with three elements, each are summary of total data, data with label 0, and data with label 1"""
    model_summeries = []

    for feature_no in range(len(featured)):
        labelzero = summarize_gaussian(featured[feature_no][0])
        labelone = summarize_gaussian(featured[feature_no][1])
        total = labelzero + labelone
        labeltotal = summarize_gaussian(total)
        model_summeries.append([labeltotal, labelzero, labelone])
    
    return model_summeries

In [ ]:
#example of input when training model
# featured_example = [[['values of feature #1 when label = 0'], ['values of feature #1 when label = 1']], [['values of feature #2 when label = 0'], ['values of feature #2 when label = 1']], ['...']]

In [ ]:
# modelmaking

datas_example = [] #featured_example-like data

summeries_example = summerize_multiple_features(datas_example)

In [ ]:
# predicting single data

input_to_predict = ['value of feature 1', 'value of feature 2', '...'] # values of feature here

if len(input_to_predict) != len(summeries_example):
    print("Bayes: number of feature does not match!")

perchance = []

for featurenum in summeries_example:
    perchance.append(calculate_class_probabilities(input_to_predict[featurenum], summeries[featurenum][0], summeries[featurenum][1], summeries[featurenum][2]))

print(thatfunctionthatconsiderspossibilityofmultiplefeatureandchoosetheclasswhichgivendatabelongstoandalsohascomicallylongname(perchance))

In [ ]:
def predict(data: list, summary: list) -> int:
    """Bayes: predict the label of single sets of data when summary"""
    
    if len(data) != len(summary):
        print("Bayes: number of feature does not match!")
        return -1

    perchance = []

    for featurenum in summary:
        perchance.append(calculate_class_probabilities(input_to_predict[featurenum], summary[featurenum][0], summary[featurenum][1], summary[featurenum][2]))

    return thatfunctionthatconsiderspossibilityofmultiplefeatureandchoosetheclasswhichgivendatabelongstoandalsohascomicallylongname(perchance)

In [ ]:
def predict_multiple(datas: list, summary: list) -> list:
    """Bayes: predict multiple datas and return the values as list"""
    
    predictions = []
    
    for data in datas:
        predictions.append(predict(data, summary))
        
    return predictions

In [ ]:
def calculate_f1(original_label: list, predicted_label: list) -> float, float, float, float:
    if len(original_label) != len(predicted_label):
        return -1
    
    # positive = 1 / negative = 0
    
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    
    for i in range(len(original_label)):
        if original_label[i] == predicted_label[i]:
            if predicted_label[i] == 1:
                TP += 1
            else:
                TN += 1
        else:
            if predicted_label[i] == 1:
                FP += 1
            else:
                FN += 1
                
    pre = TP / (TP + FP)
    rec = TP / (TP + FN)
    acc = (TP + TN) / (TP + FN + FP + TN)
    f1 = 2 * pre * rec / (pre + rec)
    
    print(f"precision: {pre}")
    print(f"recall: {rec}")
    print(f"accuracy: {acc}")
    print(f"f1-score: {f1}")
    
    return pre, rec, acc, f1

In [ ]:
feature_set_1_data = []

answersheet = [0] * len(feature_set_1_data[0][0]) + [1] * len(feature_set_1_data[0][1])

summeries = summerize_multiple_features(feature_set_1_data)

calculate_f1(answersheet, predict_multiple(feature_set_1_data, summeries))

In [ ]:
feature_set_2_data = []

answersheet2 = [0] * len(feature_set_2_data[0][0]) + [1] * len(feature_set_2_data[0][1])

summeries2 = summerize_multiple_features(feature_set_2_data)

calculate_f1(answersheet2, predict_multiple(feature_set_2_data, summeries2))